# Model Deployment

This notebook demonstrates the deployment of computer vision models to production using Databricks Model Serving.

In [ ]:
# Install dependencies
!pip install databricks-sdk mlflow

In [ ]:
# Import required modules
from databricks.sdk import WorkspaceClient
from deployment.ci_cd.deployment_pipeline import DeploymentPipeline
from deployment.serving.model_server import ModelServer
import mlflow
import json
import time

## Initialize Deployment Pipeline

Set up the deployment pipeline with necessary configurations.

In [ ]:
# Initialize workspace client
workspace = WorkspaceClient(
    host="https://your-workspace.cloud.databricks.com",
    token="your-token"
)

# Initialize deployment pipeline
pipeline = DeploymentPipeline(
    workspace_url="https://your-workspace.cloud.databricks.com",
    token="your-token",
    model_name="cv_model",
    experiment_name="cv_experiment"
)

## Define Deployment Configuration

Set up deployment parameters and thresholds.

In [ ]:
# Define metrics threshold
metrics_threshold = {
    "accuracy": 0.9,
    "precision": 0.9,
    "recall": 0.9,
    "f1": 0.9
}

# Define deployment configuration
deployment_config = {
    "endpoint_name": "cv_model_endpoint",
    "instance_type": "Standard_DS3_v2",
    "min_instances": 1,
    "max_instances": 5,
    "scale_to_zero": True
}

## Run Deployment Pipeline

Execute the deployment pipeline with the specified configuration.

In [ ]:
# Run deployment pipeline
result = pipeline.run_pipeline(
    metrics_threshold=metrics_threshold,
    endpoint_name=deployment_config["endpoint_name"]
)

print("Deployment completed!")
print(f"Endpoint name: {result['endpoint_name']}")
print(f"Model version: {result['model_version']}")
print(f"Deployment time: {result['deployment_time']}")

## Initialize Model Server

Set up the model server for monitoring and management.

In [ ]:
# Initialize model server
server = ModelServer(
    workspace_url="https://your-workspace.cloud.databricks.com",
    token="your-token",
    model_name="cv_model",
    model_version=result['model_version']
)

## Monitor Endpoint

Set up monitoring for the deployed endpoint.

In [ ]:
def monitor_endpoint(endpoint_name: str, duration: int = 300):
    """Monitor endpoint metrics for specified duration."""
    metrics_history = []
    start_time = time.time()
    
    while time.time() - start_time < duration:
        metrics = server.monitor_endpoint(endpoint_name)
        metrics_history.append(metrics)
        time.sleep(10)  # Poll every 10 seconds
    
    return metrics_history

# Start monitoring
print("Starting endpoint monitoring...")
metrics_history = monitor_endpoint(result['endpoint_name'])

# Plot metrics
plt.figure(figsize=(12, 6))
plt.plot([m['total_requests'] for m in metrics_history], label='Total Requests')
plt.plot([m['average_latency'] for m in metrics_history], label='Average Latency')
plt.plot([m['error_rate'] for m in metrics_history], label='Error Rate')
plt.xlabel('Time (10s intervals)')
plt.ylabel('Value')
plt.legend()
plt.title('Endpoint Metrics Over Time')
plt.show()

## Test Endpoint

Perform test inference on the deployed endpoint.

In [ ]:
def test_endpoint(endpoint_name: str, test_data: list):
    """Test endpoint with sample data."""
    results = []
    
    for data in test_data:
        response = server.invoke_endpoint(endpoint_name, data)
        results.append(response)
    
    return results

# Load test data
test_data = torch.load("/dbfs/path/to/test_data.pt")
test_samples = test_data[:5]  # Test with 5 samples

# Run inference
print("Running test inference...")
results = test_endpoint(result['endpoint_name'], test_samples)

# Display results
for i, (data, result) in enumerate(zip(test_samples, results)):
    print(f"\nSample {i+1}:")
    print(f"Input shape: {data.shape}")
    print(f"Prediction: {result}")

## Cleanup (Optional)

Clean up resources if needed.

In [ ]:
# Delete endpoint if needed
# server.delete_endpoint(result['endpoint_name'])